In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from scipy.stats import uniform, randint

############原始数据运用机器学习方法

In [7]:
vortex_v=pd.read_excel('C:\\Users\\Desktop\\identification1-1toZH.xlsx',sheet_name='Sheet1')
vortex_v=vortex_v.dropna()
# vortex_u=vortex_v['Ū']
# vortex_t=vortex_v['t']
print(vortex_v.columns)
# target=pd.read_excel('C:\\Users\\Desktop\\identification1-1toZH.xlsx',sheet_name='Sheet2')
# print(target)
# vortex_v_norm.plot.scatter(x='Ū',y='t')

Index(['Ū2', 'H1', 'H2', 'T1', 'T2', 't-', 'Fmin', 'height(m)', 'label'], dtype='object')


In [28]:
import warnings
from sklearn.model_selection import cross_val_score, KFold
warnings.filterwarnings('ignore')
# 划分训练集和测试集
vortex = vortex_v[['Ū2','H1','H2','T1','T2','Fmin']]
target = vortex_v[['label']]
# print(vortex.to_numpy)
# print(target.to_numpy)
# print(vortex)
# print(target)

X_train, X_test, y_train, y_test = train_test_split(vortex, target, test_size=0.2, random_state=42)

# # 定义超参数的搜索范围
# param_dist = {'C': uniform(loc=0, scale=10),
#               'kernel': ['linear','sigmoid', 'rbf'],
#               'gamma': uniform(loc=0, scale=10),
#               'max_iter': [1000000] }

# 创建SVM分类器
svm = SVC()

# # 使用随机搜索进行超参数调整
# random_search = RandomizedSearchCV(svm, param_distributions=param_dist, n_iter=10, cv=3)
# random_search.fit(X_train, y_train)

# # 输出最佳超参数组合
# print("最佳超参数组合:", random_search.best_params_)

# # 在测试集上评估模型
# best_model = random_search.best_estimator_
# test_predictions = best_model.predict(X_test)
# test_accuracy = accuracy_score(y_test, test_predictions)
# print("测试集准确率:", test_accuracy)

# 创建 k 折交叉验证对象
kfold = KFold(n_splits=10)

# 执行交叉验证
scores = cross_val_score(svm,vortex, target, cv=kfold)

# 输出交叉验证结果
print("Cross-validation scores:", scores)
print("Average Accuracy:", scores.mean())

Cross-validation scores: [0.54166667 0.70833333 0.45833333 0.54166667 0.58333333 0.79166667
 0.60869565 0.7826087  0.56521739 0.60869565]
Average Accuracy: 0.6190217391304348


#可视化SVM结果

###########逻辑斯蒂函数

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.model_selection import cross_val_score
# 创建逻辑斯蒂回归模型
model = LogisticRegression()

# 定义要调整的超参数分布
param_dist = {'C': uniform(loc=0, scale=4), 'penalty': ['l1', 'l2']}

# # 执行随机搜索
# random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10, cv=5)
# random_search.fit(X_train, y_train)

# # 输出最佳参数组合
# print("Best Parameters:", random_search.best_params_)

# # 输出最佳模型的性能
# print("Best Score:", random_search.best_score_)
# 执行交叉验证
scores = cross_val_score(model, X_train, y_train, cv=10)

# 输出每折交叉验证的准确率
print("Cross-validation scores:", scores)

# 输出平均准确率
print("Average Accuracy:", scores.mean())

Cross-validation scores: [0.78947368 0.78947368 0.68421053 0.68421053 0.68421053 0.89473684
 0.89473684 1.         0.83333333 0.88888889]
Average Accuracy: 0.814327485380117


In [34]:
import torch
import torch.nn as nn
import torch.optim as optim

vortex_np = np.array(vortex)
target_np = np.array(target)
vortex_tensor = torch.from_numpy(vortex_np).to(torch.float32)
target_tensor = torch.from_numpy(target_np).to(torch.float32)

# print(vortex_tensor.shape)
# print(target_tensor.shape)

X_train, X_test, y_train, y_test = train_test_split(vortex_tensor, target_tensor, test_size=0.2, random_state=42)

# print(X_train.dtype)
# print(X_test.dtype)

# print(X_test)
# print(y_test)

# 定义神经网络模型
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.hidden1 = nn.Linear(6,8)
        self.bn = nn.BatchNorm1d(8)

        self.hidden2 = nn.Linear(8,8)
        
        self.sigmoid = nn.Sigmoid()
        self.out = nn.Linear(8,1)

    def forward(self, x):
        out = self.hidden1(x)
        out = self.bn(out)
        out = self.sigmoid(out)
        # print('------------------')
        # print(out)
        out = self.hidden2(out)
        out = self.bn(out)
        out = self.sigmoid(out)
        # print('-------------')
        # print(out)
        out = self.out(out)
        out = self.sigmoid(out)
        # print('-------------end')
        # print(out)
        
        return out

# 创建模型实例
model = SimpleNet()

# 定义损失函数和优化器
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.02)

# 训练模型
num_epochs = 100
for epoch in range(num_epochs):
    # 前向传播
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # 反向传播和优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 打印训练信息
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



# 使用训练好的模型进行预测
with torch.no_grad():
    predictions = model(X_test)
    rounded_predictions = torch.round(predictions)

# 打印预测结果
sum_1=0
for i in range(len(X_test)):
    # print(X_test[i].numpy(), rounded_predictions[i].item())
    sum_1+=(int(rounded_predictions[i].item())==y_test[i])
    # print(predictions[i].item())
print('accuracy_score=',sum_1/len(X_test))

Epoch [10/100], Loss: 0.6425
Epoch [20/100], Loss: 0.6008
Epoch [30/100], Loss: 0.5584
Epoch [40/100], Loss: 0.5149
Epoch [50/100], Loss: 0.4524
Epoch [60/100], Loss: 0.4084
Epoch [70/100], Loss: 0.3776
Epoch [80/100], Loss: 0.3576
Epoch [90/100], Loss: 0.3421
Epoch [100/100], Loss: 0.3269
accuracy_score= tensor([0.7292])


##########点二列相关性分析

In [24]:
import scipy.stats as stats
# stats.pointbiserialr(vortex['Fmin'],target)
print(stats.pointbiserialr(vortex['Ū2'],target))
print(stats.pointbiserialr(vortex['H1'],target))
print(stats.pointbiserialr(vortex['H2'],target))
print(stats.pointbiserialr(vortex['T1'],target))
print(stats.pointbiserialr(vortex['T2'],target))
print(stats.pointbiserialr(vortex['Fmin'],target))





PointbiserialrResult(correlation=array([0.12358837382501103], dtype=object), pvalue=0.05798727523752299)
PointbiserialrResult(correlation=array([-0.46153227875060715], dtype=object), pvalue=7.488220033915613e-14)
PointbiserialrResult(correlation=array([-0.2487833336229099], dtype=object), pvalue=0.00011223585019262065)
PointbiserialrResult(correlation=array([0.22936407308445622], dtype=object), pvalue=0.00038184448364243236)
PointbiserialrResult(correlation=array([0.342133405807397], dtype=object), pvalue=6.981864859642022e-08)
PointbiserialrResult(correlation=array([-0.06952520968846497], dtype=object), pvalue=0.2874722222398788)


In [16]:
print(vortex['Fmin'])

0      0.323542
1      0.170575
2      0.367358
3      0.228615
4      0.309062
         ...   
244    0.322943
245    0.257386
246    0.364673
247    0.081005
248    0.069273
Name: Fmin, Length: 236, dtype: float64
